In [20]:
import os
import csv
from supabase import create_client, Client
import os
from dotenv import load_dotenv
from pathlib import Path

env_path = Path('..') / '.env.local'
load_dotenv(dotenv_path=env_path)

url = os.getenv("NEXT_PUBLIC_SUPABASE_URL")
key = os.getenv("NEXT_PUBLIC_SUPABASE_ANON_KEY")
supabase: Client = create_client(url, key)

data = supabase.table("league").select("*").execute()
print(data)

data=[{'league_unique': 'ncaambb', 'league_name': "NCAA Men's Basketball", 'sport': 'basketball', 'womens': False}] count=None


In [38]:
def update_game_schedule(game_schedule_csv):
    game_insert = []
    with open(game_schedule_csv, 'r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        # next(reader)
        for row in reader:
            print(row)
            if row != {'game_date': '', 'team_2_id': '', 'team_1_id': '', 'game_time': '', 'round_num': '', 'competition_id': ''}:
                game_insert.append(row)

    data = supabase.table("game").upsert(game_insert,
                                         # ignore_duplicates=True, on_conflict="game_date, team_1_id, team_2_id"
                                         ).execute()
    print(data)

In [39]:
round = "3"
update_game_schedule(f"game-schedule-round-{round}-2023-ncaa-tournament.csv")

{'game_date': '2023-03-18', 'team_2_id': 'san-diego-state', 'team_1_id': 'furman', 'game_time': '12:30', 'round_num': '3', 'competition_id': '1'}
{'game_date': '2023-03-18', 'team_2_id': 'tennessee', 'team_1_id': 'duke', 'game_time': '14:40', 'round_num': '3', 'competition_id': '1'}
{'game_date': '2023-03-18', 'team_2_id': 'kansas', 'team_1_id': 'arkansas', 'game_time': '17:15', 'round_num': '3', 'competition_id': '1'}
{'game_date': '2023-03-18', 'team_2_id': 'missouri', 'team_1_id': 'princeton', 'game_time': '18:10', 'round_num': '3', 'competition_id': '1'}
{'game_date': '2023-03-18', 'team_2_id': 'houston', 'team_1_id': 'auburn', 'game_time': '19:10', 'round_num': '3', 'competition_id': '1'}
{'game_date': '2023-03-18', 'team_2_id': 'texas', 'team_1_id': 'penn-state', 'game_time': '19:45', 'round_num': '3', 'competition_id': '1'}
{'game_date': '2023-03-18', 'team_2_id': 'ucla', 'team_1_id': 'northwestern', 'game_time': '20:40', 'round_num': '3', 'competition_id': '1'}
{'game_date': '2

In [59]:
import pandas as pd
import datetime
def generate_game_scoring_sheet(competition_id: int = 1, date = datetime.date.today()):
    data = supabase.table("players_in_games_view").select("*").eq("game_date", date).eq('competition_id', competition_id).execute()
    df = pd.DataFrame(data.data)
    df['points'] = None
    df['lost'] = None
    df['inactive'] = None
    # df['rebounds'] = 0
    # df['assists'] = 0
    df.to_csv(f"{date}-game-scoring-2023-ncaa-tournament-losers.csv", index=False, columns=['game_time', 'team_unique', 'lost', 'player_unique', 'points','inactive','game_id'])
    # 'rebounds', 'assists'])

In [61]:
# date = datetime.date.today()
date = "2023-03-17"
generate_game_scoring_sheet(date=date)

In [65]:
def update_scores_from_csv(current_round: int, competition_id: int = 1, game_scoring_csv = f"{datetime.date.today()}-game-scoring-2023-ncaa-tournament.csv"):
    player_game_insert = []
    losing_teams = []
    inactive_players = []
    team_1 = ""
    team_2 = ""
    with open(game_scoring_csv, 'r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        next(reader)
        for row in reader:
            if row['lost'].upper() == 'L':
                losing_teams.append(row['team_unique'])
            if row['inactive'].upper() == 'I':
                inactive_players.append(row['player_unique'])
            if row['points'] != '':
                team_2 = team_1
                team_1 = row['team_unique']
                player_game_row = {
                    "player_unique": row['player_unique'],
                    "game_id": row['game_id'],
                    "points": row['points'],
                }
                player_game_insert.append(player_game_row)

    data = supabase.table("player_game").upsert(player_game_insert,
                                         ignore_duplicates=False, on_conflict="game_id, player_unique").execute()
    updated = supabase.table("competition_updated").insert({"competition_id": competition_id, "current_round": current_round}).execute()
    for team in losing_teams:
        loser_data = supabase.table('team_competition').update({'round_eliminated': current_round}).eq('team_unique', team).eq('competition_id', competition_id).execute()
        print("losers")
        print(loser_data)
    for player in inactive_players:
        player_data = supabase.table('player_competition').update({'inactive': True}).eq('player_unique', player).eq('competition_id', competition_id).execute()
        print("inactive players")
        print(player_data)
    print(data)

In [74]:
# date = datetime.date.today()
current_round=3
date = "2023-03-18"
update_scores_from_csv(current_round=current_round, game_scoring_csv = f"{date}-game-scoring-2023-ncaa-tournament.csv")


losers
data=[{'team_unique': 'duke', 'competition_id': 1, 'league_unique': 'ncaambb', 'seed': 5, 'overall_seed': 18, 'region': 'EAST', 'round_eliminated': 3, 'round_started': 2, 'team_stats': {'pace': 65.7, 'conference': 'acc', 'offensive_rating': 110.0, 'assist_percentage': 56.9, 'turnover_percentage': 15.4, 'strength_of_schedule': 7.03, 'free_throw_attempt_rate': 0.308, 'three_point_attempt_rate': 0.354, 'true_shooting_percentage': 0.549, 'effective_field_goal_percentage': 0.511, 'two_point_field_goal_percentage': 0.514, 'three_point_field_goal_percentage': 0.336, 'opp_effective_field_goal_percentage': 0.463}, 'stats_thru': '2023-03-12', 'team_win_loss': {'wins': 26.0, 'losses': 8.0, 'games_played': 34.0, 'conference_wins': 14.0, 'conference_losses': 6.0}}] count=None
losers
data=[{'team_unique': 'kansas', 'competition_id': 1, 'league_unique': 'ncaambb', 'seed': 1, 'overall_seed': 3, 'region': 'WEST', 'round_eliminated': 3, 'round_started': 2, 'team_stats': {'pace': 70.1, 'conference